# Gaussian Elimination

In [39]:
import numpy as np
from numpy import linalg as la

## Partial Pivoting

In [40]:
def partial_pivot(mat: np.ndarray, column: int) -> np.ndarray:
    """
    Partially pivots matrix(mat) with respect to input column.
    """
    # Should consider the absolute maximum value.
    # Should add 'column' because the argmax returns relative index.
    max_row = np.argmax(abs(mat[column:, column])) + column

    if max_row == column:
        return mat
    else:
        # How to switch rows!
        mat[[column, max_row], :] = mat[[max_row, column], :]
        return mat

In [41]:
# Example of partial pivoting
mat = np.array([[4.0, 3, -5, -7], [-2, -4, 5, 11], [8, 8, 0, -8]])
print(f"original matrix >> {mat}")
mat_pivot = partial_pivot(mat, 1)
print(f"pivoted matrix >> {mat_pivot}")

original matrix >> [[ 4.  3. -5. -7.]
 [-2. -4.  5. 11.]
 [ 8.  8.  0. -8.]]
pivoted matrix >> [[ 4.  3. -5. -7.]
 [ 8.  8.  0. -8.]
 [-2. -4.  5. 11.]]


## Row Echelon Form

In [42]:
def row_echelon_form(mat: np.ndarray, naive: bool = False) -> np.ndarray:
    """
    Returns row echelon form of a matrix.
    """
    row = len(mat)
    # Copy of matrix to return
    # Note that simply assigning mat to mat_ref will modify the original matrix too!
    # =====
    # When you assign an ndarray to another variable,
    # it creates a reference to the same data rather than making a copy of the data.
    # ===== By. ChatGPT

    mat_ref = np.copy(mat)

    for row_idx in range(row - 1):
        if ~naive:
            mat_ref = partial_pivot(mat_ref, row_idx)
        for row_idx2 in range(row_idx + 1, row):
            if mat_ref[row_idx2, row_idx] != 0:
                multiplier = mat_ref[row_idx2, row_idx] / mat_ref[row_idx, row_idx]
                mat_ref[row_idx2, :] = (
                    mat_ref[row_idx2, :] - multiplier * mat_ref[row_idx, :]
                )

    return mat_ref

In [43]:
# Example of row echelon form
mat = np.array([[4.0, 3, -5, -7], [-2, -4, 5, 11], [8, 8, 0, -8]])
mat_ref = row_echelon_form(mat)
print(f"original >> {mat}")
print(f"ref >> {mat_ref}")

original >> [[ 4.  3. -5. -7.]
 [-2. -4.  5. 11.]
 [ 8.  8.  0. -8.]]
ref >> [[ 8.   8.   0.  -8. ]
 [ 0.  -2.   5.   9. ]
 [ 0.   0.  -7.5 -7.5]]


## Get system's info

In [44]:
def get_sys_info(coeff_mat: np.ndarray, const_vect: np.ndarray) -> int:
    """
    Prints out the info for system of linear equations
    and returns number of solutions.
    """
    row, column = np.shape(coeff_mat)
    rank_coeff = la.matrix_rank(coeff_mat)
    # Augmented matrix
    aug_mat = np.hstack((coeff_mat, const_vect))
    rank_aug = la.matrix_rank(aug_mat)

    if rank_aug == rank_coeff + 1:
        print("Inconsistent system. No solution.")
        num_sol = 0
        return num_sol
    elif rank_aug == rank_coeff:
        if row == column:
            print("Consistent and independent system. Unique solution.")
            num_sol = 1
            return num_sol
        else:
            print("Consistent and dependent system. Infinite solutions.")
            num_sol = np.inf
            return num_sol
    else:
        print("Unvalid system.")
        return np.nan

In [45]:
# Example of getting system info
coeff_mat = np.array([[4.0, 3, -5], [-2, -4, 5], [8, 8, 0]])
const_vect = np.array([[-7], [11], [-8]])
num_sol = get_sys_info(coeff_mat, const_vect)
print(f"Number of solutions >> {num_sol}")

Consistent and independent system. Unique solution.
Number of solutions >> 1


## Gaussian Elimination

In [46]:
def gaussian_elim(
    coeff_mat: np.ndarray, const_vect: np.ndarray, naive: bool = False
) -> np.ndarray:
    """
    Implements Gaussian elimination.
    Returns the root of the system.
    """
    row = len(coeff_mat)
    num_sol = get_sys_info(coeff_mat, const_vect)
    if num_sol != 1:
        raise Exception("Cannot get root!")
    aug_mat_ref = row_echelon_form(np.hstack((coeff_mat, const_vect)), naive)
    root_vect = np.zeros((row, 1))
    # Backward substitution
    # Can iterate through range in reverse order.
    for row_idx in range(row - 1, -1, -1):
        root_vect[row_idx] = (
            aug_mat_ref[row_idx, -1]
            - np.dot(
                # This part should use aug_mat_ref instead of aug_mat!
                aug_mat_ref[row_idx, row_idx + 1 : -1],
                root_vect[row_idx + 1 :, 0],
            )
        ) / aug_mat_ref[row_idx, row_idx]

    return root_vect

In [47]:
# Example of Gaussian elimination
coeff_mat = np.array([[4.0, 3, -5], [-2, -4, 5], [8, 8, 0]])
const_vect = np.array([[-7], [11], [-8]])
root = gaussian_elim(coeff_mat, const_vect, False)
print(f"solution >> {root}")

Consistent and independent system. Unique solution.
solution >> [[ 1.]
 [-2.]
 [ 1.]]


## Backward Elimination & Forward Substitution

In [48]:
# Reversed partial_pivot for backward elimination.

In [49]:
def rev_partial_pivot(mat: np.ndarray, column: int) -> np.ndarray:
    """
    Partially pivots matrix(mat) with respect to input column.
    """
    # Should consider the absolute maximum value.
    # Should add 'column' because the argmax returns relative index.
    max_row = np.argmax(abs(mat[: column + 1, column]))

    if max_row == column:
        return mat
    else:
        # How to switch rows!
        mat[[column, max_row], :] = mat[[max_row, column], :]
        return mat

In [50]:
# Use backward_elim instead of row_echelon_form
def backward_elim(mat: np.ndarray, naive: bool = False) -> np.ndarray:
    """
    Makes the input matrix into lower triangluar form
    by elemetary row operation.
    """
    row = len(mat)
    mat_lowtri = np.copy(mat)

    for row_idx in range(row - 1, 0, -1):
        if ~naive:
            mat_lowtri = rev_partial_pivot(mat_lowtri, row_idx)
        for row_idx2 in range(row_idx - 1, -1, -1):
            multiplier = mat_lowtri[row_idx2, row_idx] / mat_lowtri[row_idx, row_idx]
            mat_lowtri[row_idx2, :] = (
                mat_lowtri[row_idx2, :] - multiplier * mat_lowtri[row_idx, :]
            )

    return mat_lowtri

In [57]:
def rev_gaussian_elim(
    coeff_mat: np.ndarray, const_vect: np.ndarray, naive: bool = False
) -> np.ndarray:
    """
    Reversed Gaussian elimination.
    Implements backward elimination and forward substitution.
    """
    row, column = np.shape(coeff_mat)
    num_sol = get_sys_info(coeff_mat, const_vect)
    if num_sol != 1:
        raise Exception("Cannot get root!")
    aug_mat_lowtri = backward_elim(np.hstack((coeff_mat, const_vect)), naive)
    root_vect = np.zeros((row, 1))
    # Forward substitution
    for row_idx in range(row):
        root_vect[row_idx] = (
            aug_mat_lowtri[row_idx, -1]
            - np.dot(aug_mat_lowtri[row_idx, :row_idx], root_vect[:row_idx, 0])
        ) / aug_mat_lowtri[row_idx, row_idx]

    return root_vect

In [59]:
# Example of Gaussian elimination
coeff_mat = np.array([[4.0, 3, -5], [-2, -4, 5], [8, 8, 0]])
const_vect = np.array([[-7], [11], [-8]])
root = rev_gaussian_elim(coeff_mat, const_vect, False)
print(f"solution >> {root}")

Consistent and independent system. Unique solution.
solution >> [[ 1.]
 [-2.]
 [ 1.]]
